# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [1]:
# Dependencies and Setup

import pandas as pd
import requests


In [2]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [3]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [4]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

SECTION 2     
PowerBreakdown data transformation

In [5]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [6]:
df_US

,0
region,US-SW-PNM
datetime,2024-08-13T15:00:00.000Z
nuclear,0
geothermal,0
biomass,0
coal,157
wind,11
solar,680
hydro,15
gas,401


In [7]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_16120\3249227210.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1335,1344,1344
unique,53,24,298,23,64,543,428,381,507,892,39,271,2,27,85,1073,2
top,US-CAL-BANC,2024-08-13T15:00:00.000Z,0,0,0,0,0,0,0,0,0,0,0,0,100,160,False
freq,48,56,857,1320,1208,599,707,708,334,193,1243,622,1343,1300,183,9,936


In [8]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 'renewable_percentage':0,
           'battery_discharge':0, 'total_consumption':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000
mean,1125.617560,12.984375,18.643601,1431.622768,849.699405,508.525298,532.423363,3585.172619,1.627232,105.924107,0.216518,20.866071,31.968750,8193.324405
std,2529.610319,96.537072,94.541297,3762.613803,3002.626338,2183.856555,1121.922482,7447.696728,7.970358,366.817224,7.937676,325.727691,33.686133,16049.468201
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,281.000000,0.000000,0.000000,0.000000,0.000000,5.000000,888.500000
50%,0.000000,0.000000,0.000000,159.000000,0.000000,0.000000,65.500000,1034.000000,0.000000,3.000000,0.000000,0.000000,18.000000,2229.000000
75%,132.000000,0.000000,0.000000,1208.000000,241.500000,142.000000,542.250000,2289.250000,0.000000,60.000000,0.000000,0.000000,51.000000,6786.000000
max,14197.000000,780.000000,604.000000,29608.000000,21141.000000,19269.000000,8084.000000,47451.000000,104.000000,8215.000000,291.000000,6979.000000,100.000000,101027.000000


In [9]:
# convert measured Energy values to integers in Giga Watts
#convert_dict = {'hydro': int}
convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
              'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                  object
datetime                object
nuclear                  int32
geothermal               int32
biomass                  int32
coal                     int32
wind                     int32
solar                    int32
hydro                    int32
gas                      int32
oil                      int32
unknown                  int64
hydro-discharge          int32
battery_discharge        int32
renewable_percentage     int32
total_consumption        int32
estimated               object
dtype: object

In [10]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [11]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2024-08-13 15:00:00,US-SW-PNM,0,0,0,157,11,680,15,401,0,0,0,0,56,1264,False,15:00:00,2024-08-13
2024-08-13 16:00:00,US-SW-PNM,0,0,0,159,7,684,15,383,0,0,0,0,57,1248,False,16:00:00,2024-08-13
2024-08-13 17:00:00,US-SW-PNM,0,0,0,158,32,807,14,384,0,0,0,0,61,1395,False,17:00:00,2024-08-13
2024-08-13 18:00:00,US-SW-PNM,0,0,0,158,234,804,14,411,0,0,0,0,65,1620,True,18:00:00,2024-08-13
2024-08-13 19:00:00,US-SW-PNM,0,0,0,158,435,801,13,437,0,0,0,0,68,1845,True,19:00:00,2024-08-13


Section 3
Transform carbon intensity data

In [12]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [13]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_16120\3332989139.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [14]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [15]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [16]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-08-13 15:00:00,US-SW-PNM,0,0,0,157,11,680,15,401,0,0,0,0,56,1264,False,15:00:00,2024-08-13,350,False
2024-08-13 16:00:00,US-SW-PNM,0,0,0,159,7,684,15,383,0,0,0,0,57,1248,False,16:00:00,2024-08-13,339,False
2024-08-13 17:00:00,US-SW-PNM,0,0,0,158,32,807,14,384,0,0,0,0,61,1395,False,17:00:00,2024-08-13,338,False
2024-08-13 18:00:00,US-SW-PNM,0,0,0,158,234,804,14,411,0,0,0,0,65,1620,True,18:00:00,2024-08-13,304,False
2024-08-13 19:00:00,US-SW-PNM,0,0,0,158,435,801,13,437,0,0,0,0,68,1845,True,19:00:00,2024-08-13,272,True


In [17]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/allRegions/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_16120\2571967791.py:2: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')


,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36387.000000,36387.000000,36387.000000,36387.000000,36289.000000,36349.000000,36377.000000
mean,1193.418781,14.329274,20.841246,1591.776782,652.788465,502.950876,592.392968,4050.007147,3.456509,97.321076,0.378817,18.595130,32.104853,8745.016534,453.046430
std,2540.070490,102.622047,102.872742,4187.098407,2362.107330,2055.486626,1315.430911,8139.579284,32.003886,294.953640,10.722478,278.851896,34.612709,16584.903368,200.780861
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,0.000000,0.000000,0.000000,0.000000,4.000000,764.000000,319.000000
50%,0.000000,0.000000,0.000000,24.000000,0.000000,0.000000,73.000000,972.000000,0.000000,8.000000,0.000000,0.000000,18.000000,2283.000000,408.000000
75%,649.000000,0.000000,0.000000,1239.000000,184.000000,157.000000,630.000000,2858.000000,0.000000,76.000000,0.000000,0.000000,50.000000,8504.000000,557.000000
max,14865.000000,819.000000,2416.000000,49357.000000,25406.000000,20227.000000,65867.000000,79396.000000,2054.000000,13129.000000,557.000000,7399.000000,100.000000,128873.000000,1250.000000


In [18]:
df_us_energy_reindex.drop_duplicates(['UTC date', 'UTC time', 'region'],keep='last', inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36377.000000,36387.000000,36387.000000,36387.000000,36387.000000,36289.000000,36349.000000,36377.000000
mean,1193.418781,14.329274,20.841246,1591.776782,652.788465,502.950876,592.392968,4050.007147,3.456509,97.321076,0.378817,18.595130,32.104853,8745.016534,453.046430
std,2540.070490,102.622047,102.872742,4187.098407,2362.107330,2055.486626,1315.430911,8139.579284,32.003886,294.953640,10.722478,278.851896,34.612709,16584.903368,200.780861
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,0.000000,0.000000,0.000000,0.000000,4.000000,764.000000,319.000000
50%,0.000000,0.000000,0.000000,24.000000,0.000000,0.000000,73.000000,972.000000,0.000000,8.000000,0.000000,0.000000,18.000000,2283.000000,408.000000
75%,649.000000,0.000000,0.000000,1239.000000,184.000000,157.000000,630.000000,2858.000000,0.000000,76.000000,0.000000,0.000000,50.000000,8504.000000,557.000000
max,14865.000000,819.000000,2416.000000,49357.000000,25406.000000,20227.000000,65867.000000,79396.000000,2054.000000,13129.000000,557.000000,7399.000000,100.000000,128873.000000,1250.000000


In [19]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(['UTC date', 'UTC time', 'region'],keep='first', inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,37278.000000,37278.00000,37278.000000,37278.000000,37278.000000,37278.000000,37278.000000,37278.000000,37288.000000,37288.000000,37288.000000,37288.000000,37190.00000,37250.000000,37278.000000
mean,1193.389962,14.32719,20.824052,1586.511535,660.510408,496.725844,590.990477,4039.487499,3.414477,97.647983,0.369663,18.894443,32.06448,8729.678282,453.912576
std,2541.655882,102.62474,102.807467,4175.263405,2391.587646,2043.454403,1311.387760,8122.623836,31.637935,298.583399,10.592297,282.306459,34.61121,16563.933227,199.376122
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,113.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,238.000000,0.000000,0.000000,0.000000,0.000000,4.00000,763.000000,321.000000
50%,0.000000,0.00000,0.000000,25.000000,0.000000,0.000000,72.000000,971.000000,0.000000,8.000000,0.000000,0.000000,18.00000,2278.000000,411.000000
75%,649.000000,0.00000,0.000000,1230.000000,185.000000,152.000000,627.000000,2839.000000,0.000000,76.000000,0.000000,0.000000,50.00000,8470.000000,559.000000
max,14865.000000,819.00000,2416.000000,49357.000000,25406.000000,20227.000000,65867.000000,79396.000000,2054.000000,13129.000000,557.000000,7399.000000,100.00000,128873.000000,1250.000000


In [20]:
df_both.to_csv(f'data/allRegions/runningUSenergy_data.csv')

In [21]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,9418.000000,9418.000000,9418.000000,9418.000000,9418.000000,9418.000000,9418.000000,9418.000000,9418.000000,9418.000000,9418.0,9418.000000,9320.000000,9402.000000,9418.000000
mean,1513.164791,52.929284,75.874602,1848.565513,1295.603950,735.736568,1029.677426,5497.000212,8.192185,106.141431,0.0,72.693778,32.805043,12256.487981,447.375027
std,2345.700622,192.053865,184.265672,4314.802138,3342.618592,2839.792899,1676.038932,7864.775693,58.198260,225.678043,0.0,554.653494,31.468703,16821.890111,187.690910
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,254.250000,0.000000,0.000000,0.0,0.000000,8.000000,908.250000,319.000000
50%,5.000000,0.000000,0.000000,35.000000,48.000000,0.000000,220.000000,1491.000000,0.000000,8.000000,0.0,0.000000,22.000000,3777.500000,416.000000
75%,2263.000000,0.000000,9.000000,1172.750000,565.750000,136.000000,1333.000000,9225.750000,0.000000,92.000000,0.0,0.000000,49.000000,19294.500000,542.000000
max,14865.000000,818.000000,2416.000000,35359.000000,25406.000000,20227.000000,65867.000000,79396.000000,2054.000000,2053.000000,0.0,7399.000000,100.000000,117668.000000,1250.000000


In [22]:
df_both_cleaned.to_csv(f'data/runningUSenergy_data_filtered.csv')